<img src="https://imgur.com/3U3hI1u.png" width="100%" />

#### Boas vindas a sua segunda e última tarefa de Aprendizado por Reforço!

Neste exercício, você deverá implementar e comparar diferentes algoritmos de **Aprendizado por Reforço Profundo** utilizando a biblioteca _[Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/)_.

A _Stable Baselines_ é uma biblioteca de Aprendizado por Reforço que implementa diversos algoritmos de agentes, além de várias funcionalidades úteis para seu treinamento. Suas implementações são bem simples e intuitivas, mas sem deixarem de ser otimizadas e poderosas, buscando facilitar o desenvolvimento de projetos de reforço de alta qualidade.

Antes de começar a tarefa, é importante acessar e se familiarizar com o tutorial da biblioteca disponível neste repositório! Depois de rodar o guia, você já estará capaz de completar este trabalho.

## Escolha do Ambiente

Antes de analisar o possíveis algoritmos, o primeiro passo é escolher qual ambiente você quer resolver! Para esta tarefa, separamos dois possíveis ambientes diferentes, em ordem de dificuldade, que você poderá escolher: **CartPole** e **Pendulum**. Lembrando que, quanto mais difícil um ambiente, mais demorado será o treinamento.

A seguir, estão as descrições de cada um deles:

<h2 align="center">CartPole</h2>
<img src="https://bytepawn.com/images/cartpole.gif" width=50% />

**CartPole** é o ambiente de Aprendizado por Reforço mais comum do Gym, no qual deve-se balancear um pêndulo invertido conectado a um carrinho, somente controlando os movimentos do carrinho.

### Características do Ambiente

O **Espaço de Observação** do CartPole é definido por 4 informações:

<br>

|     | Informação                         | Min     | Max    |
| :-- | :--------------------------------- | :-----: | :----: |
| 0   | Posição do Carrinho                | -4.8    | 4.8    |
| 1   | Velocidade do Carrinho             | -Inf    | Inf    |
| 2   | Ângulo da Barra                    | -24 deg | 24 deg |
| 3   | Velocidade na Extremidade da Barra | -Inf    | Inf    |

<br>

A posição do carrinho vai de -4.8 a 4.8, mas ele perde o episódio caso saia dos limites de -2.4 e 2.4. Da mesma forma, o ângulo da barra vai de -24° a 24°, porém o episódio acaba caso a barra saia dos limites de -12° e 12°.

Já o **Espaço de Ação** é composto por duas ações únicas: mover o carrinho para a **esquerda** ou para a **direita**.

Quando queremos mover o carrinho para a esquerda, fazemos um `env.step(0)`; quando queremos movê-lo para a direita, enviamos um `env.step(1)`

| Ação | Significado           |
| :--- | :-------------------- |
| 0    | Mover para a esquerda |
| 1    | Mover para a direito  |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, que é igual a +1 para cada instante que passa sem o agente perder. Assim, o CartPole é incentivado a sobreviver por mais tempo.

Para testar o modelo, vamos precisar de 2 bibliotecas: **gym** (para inicialização dos ambientes) e **stable_baselines3** (para inicialização e avaliação dos modelos):

In [ ]:
import gym 
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

# Definindo ambiente
env = gym.make("CartPole-v1")

# Definindo modelo
model = PPO("MlpPolicy", env, seed=1, verbose=1)

# Avaliando o agente
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5, deterministic=True)

print(f"Recompensa Média: {mean_reward:.2f} +/- {std_reward}")

<h2 align="center">Pendulum</h2>

<img src="https://www.gymlibrary.ml/_static/videos/classic_control/pendulum.gif" width=30% />

**Pendulum** é um ambiente do Gym que simula um pêndulo pendurado por um ponto tentando se balancear de cabeça para baixo. O agente deve um torque no pêndulo de forma que ele se levante e fique parado em pé.

### Características do Ambiente

O **Espaço de Observação** do ambiente é definido por 8 informações.

| Estado    | Informação                                     |
| :-------- | :--------------------------------------------- |
| 0         | Posição no eixo _x_ da ponta do pêndulo        |
| 1         | Posição no eixo _y_ da ponta do pêndulo        |
| 2         | Velocidade angular do pêndulo                  |

Já o **Espaço de Ação** é um espaço **contínuo** do torque aplicado no pêndulo.

| Ação | Significado     | Intervalo   |
| :--- | :-------------- | :---------- |
| 0    | Torque          | $-2$ a $+2$ |

Por fim, cada vez que tomamos uma ação, recebemos do ambiente uma **recompensa**, que segue a seguinte equação:

$$r = -(\theta{}^2 + 0.1 * \dot{\theta}^2 + 0.001 * \tau{})$$

Desta forma, temos que a recompensa é menor quando o ângulo do pêndulo é menor (mais em pé), quando a velocidade angular é baixa, e quando usamos pouco torque.

Por ser um ambiente com espaço de ação contínuo, os algoritmos que podemos usar serão diferentes, e o treinamento pode ser mais demorado.

### Ambiente

Para criar o ambiente, basta rodar a linha de código a seguir:

In [ ]:
env = gym.make("Pendulum-v1")

### Exercício 1 - Testando Modelos

Caro piloto, agora que você conhece esses dois ambientes, é hora de brincar com eles. Você deverá testar diferentes algoritmos (a seu critério), e ver sua recompensa média. Para ver quais as limitações dos modelos, veja esse [link](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html). Abaixo, criamos uma função que será útil para comparar os modelos posteriormente.

In [ ]:
def getValues(model, env, n_episodes, info_dict):
    model_name = str(model.__class__).split(".")[-1][:-2]

    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_episodes, deterministic=True)
    
    info_dict[model_name] = {}
    info_dict[model_name]["mean_reward"] = mean_reward
    info_dict[model_name]["std_reward"] = std_reward
    
    return info_dict

algorithms_dict = {}

#### Treinando e Avaliando seu próprio modelo

Primeiramente, agora você deve decidir em qual ambiente você deseja treinar seu agente. Para isto, basta tirar o comentário da linha referente ao ambiente escolhido:

In [ ]:
# env_name = "CartPole-v1"
# env_name = "Pendulum-v1"

Em seguida, você está livre para testar diferentes algoritmos para seu ambiente!

In [ ]:
from stable_baselines3 import ... # Importe o modelo do stable_baselines3

# Definindo o ambiente
env = gym.make(env_name)

model = ... # Defina o modelo
model.learn(total_timesteps=...) # Treine o modelo
n_episodes = ... # Defina o número de episódios

# Avaliando o agente e guardando o desempenho no dicionário
algorithms_dict = getValues(model, env, n_episodes, algorithms_dict)

In [ ]:
### Espaço livre para testagem de diferentes algoritmos


A seguir, detalhe um pouco mais quais foram os algoritmos testados bem como a performance obtida por cada um.

Este detalhamento pode ser feito por meio de um ou mais gráficos mostrando o desempenho dos modelos, ou simplesmente por texto.

In [ ]:
### Espaço para o Piloto criar gráficos ou textos para mostrar os diferentes resultados entre modelos


### Escolha do Algoritmo

Após testar e analisar diversos algoritmos diferentes, qual foi o escolhido?

_Pergunta Extra:_ você usou algum critério para escolher quais algoritmos seriam testados?

In [ ]:
### Qual foi o algoritmo escolhido?
